# Kladd for sentimentanalys
INPUT: Kollokasjonsdf 
OUTPUT: Positiv score, Negativ score, Nøytral score


# Totalstatistikk
INPUT: korpus aviser fra 2000-2022 der en form av "barnevern" forekommer 

OUTPUT: CSV
kolonner: dato, avisnavn, frekvens av "barnevern" i utgivelsen, sentimentscore = {positiv:int, negativ:int, nøytral: int)} 
e

- [ ] 

In [143]:
import dhlab as dh
import pandas as pd

In [144]:
# Data import 
positive_tokens = pd.read_csv("/home/larsm/projects/dhlab_projects/news_sentiment/sentimentanalyse/norsentlex/Fullform/Fullform_Positive_lexicon.txt", names=["terms"])
negative_tokens = pd.read_csv("/home/larsm/projects/dhlab_projects/news_sentiment/sentimentanalyse/norsentlex/Fullform/Fullform_Negative_lexicon.txt", names=["terms"])

In [145]:
positive_tokens

,terms
0,absolutt
1,absolutta
2,absolutte
3,absoluttene
4,absolutter
...,...
6098,ønskete
6099,ønskt
6100,ønskte
6101,øyeblikkelig


In [146]:
negative_tokens

,terms
0,abnorm
1,abnorme
2,abnormt
3,abort
4,aborten
...,...
14834,ørlille
14835,ørlita
14836,ørlite
14837,ørliten


In [147]:
def count_sentiment_tokens(coll, terms):     
    
    target_terms = terms.join(coll, how="inner", on="terms")
    
    return target_terms

In [177]:
def outer(coll, pos, neg):
    coll.index = coll.index.str.lower()
    coll.index.name = "index"
    coll = coll.groupby('index')["counts"].sum().to_frame("counts")    
    assert coll.index.is_unique
    
    positive_counts = count_sentiment_tokens(coll, pos)
    negative_counts = count_sentiment_tokens(coll, neg)
    #neutral_counts = coll.index.str.isin(positive_counts.index + negative_counts.index)
    duplicates = pd.concat([coll, negative_counts, positive_counts], axis=1).index.drop_duplicates(keep=False).to_series("toks")
    neutral_counts = coll.join(duplicates, how="inner", on="toks")
    
    
    positive_counts["sentiment"] = "pos"
    negative_counts["sentiment"] = "neg"
    neutral_counts["sentiment"] = "neut"
    
    return pd.concat([positive_counts, negative_counts, neutral_counts], axis=1)

In [181]:
pos = positive_tokens
neg = negative_tokens

In [182]:
coll.index = coll.index.str.lower()
coll.index.name = "index"
coll = coll.groupby('index')["counts"].sum().to_frame("counts")    
assert coll.index.is_unique

positive_counts = count_sentiment_tokens(coll, pos)
negative_counts = count_sentiment_tokens(coll, neg)
#neutral_counts = coll.index.str.isin(positive_counts.index + negative_counts.index)
duplicates = pd.concat([coll, negative_counts, positive_counts], axis=1).index.drop_duplicates(keep=False).to_series("toks")
neutral_counts = coll.join(duplicates, how="inner", on="toks")


positive_counts["sentiment"] = "pos"
negative_counts["sentiment"] = "neg"
neutral_counts["sentiment"] = "neut"

TypeError: Index(...) must be called with a collection of some kind, 'toks' was passed

In [ ]:
 +

In [211]:
for term in coll.iterrows():
    #print(term[0] )
    if term[0] not in set(positive_counts.index).union(set(negative_counts.index)):
        print(term[1])
    
    
    #break

counts    9
Name: !, dtype: int64
counts    10
Name: ", dtype: int64
counts    1
Name: #, dtype: int64
counts    4
Name: %, dtype: int64
counts    2
Name: &, dtype: int64
counts    10
Name: ', dtype: int64
counts    47
Name: (, dtype: int64
counts    49
Name: ), dtype: int64
counts    13
Name: *, dtype: int64
counts    707
Name: ,, dtype: int64
counts    175
Name: -, dtype: int64
counts    1189
Name: ., dtype: int64
counts    2
Name: ..., dtype: int64
counts    1
Name: .12, dtype: int64
counts    1
Name: .94, dtype: int64
counts    19
Name: /, dtype: int64
counts    4
Name: 0, dtype: int64
counts    1
Name: 00.20, dtype: int64
counts    3
Name: 000, dtype: int64
counts    1
Name: 02, dtype: int64
counts    1
Name: 03, dtype: int64
counts    1
Name: 04.15, dtype: int64
counts    4
Name: 06, dtype: int64
counts    19
Name: 1, dtype: int64
counts    1
Name: 1,3, dtype: int64
counts    2
Name: 1,5, dtype: int64
counts    8
Name: 1., dtype: int64
counts    1
Name: 1.000, dtype: int64
counts

In [ ]:
coll.filter(items=())

In [200]:
coll.

SyntaxError: invalid syntax (1101847456.py, line 1)

In [199]:
def 
    for term in coll.index: 
        if term not in list(negative_counts.terms).extend(list(positive_counts.terms)): 
            yield coll.loc[term]


#coll.merge([negative_counts.terms, positive_counts.terms])#.reset_index().drop_duplicates(keep=False, subset="index")#.set_index="index"

TypeError: DataFrame.join() got an unexpected keyword argument 'axis'

In [149]:
coll = c.coll("barnevernet").frame

coll.index = coll.index.str.lower()
coll.index.name = "index"
coll = coll.groupby('index')["counts"].sum().to_frame("counts")

In [172]:
coll

,counts
index,
!,9
"""",10
#,1
%,4
&,2
...,...
•,14
€,2
−,15


In [178]:
outer(coll, positive_tokens, negative_tokens)

TypeError: Index(...) must be called with a collection of some kind, 'toks' was passed

In [150]:
kladd(colldf, positive_tokens)

,terms,counts
0,absolutt,2
37,alvor,5
39,alvorlig,5
40,alvorlige,3
53,ambisjoner,1
...,...,...
6083,økte,4
6089,ønske,2
6095,ønsker,14
6096,ønskes,2


In [151]:
c = dh.Corpus(doctype="digavis", limit=10000)

In [152]:
colldf = c.coll("barnevernet").frame

In [153]:
kladd(colldf, positive_tokens)

,terms,counts
37,alvor,1
38,alvoret,1
39,alvorlig,4
40,alvorlige,2
55,anbefale,1
...,...,...
6082,økt,19
6083,økte,4
6090,ønskede,1
6095,ønsker,10


In [154]:
kladd(colldf, negative_tokens)

,terms,counts
37,advarer,1
44,advarte,1
59,alarm,1
90,alvor,1
91,alvoret,1
...,...,...
14614,voldsom,1
14647,vond,2
14649,vondt,1
14810,ødelegger,1


In [155]:
colldf.groupby(by="index")

In [156]:
c.coll("barnevernet").sort().frame

,counts
.,971
i,671
",",601
og,476
til,315
...,...
familiei,1
800,1
familiene,1
familiens,1


In [157]:
dh.Collocations(c, words="barnevernet")

,counts
!,9
"""",9
#,1
%,2
&,1
...,...
—,14
‘,1
’,2
•,18


In [158]:
c.collocations("barnevernet")

AttributeError: 'Corpus' object has no attribute 'collocations'

------


IDAG: 
- [ ] Skripte sentimentanalyse på kollokasjoner 
- [ ] Produsere CSV 